In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1)
filepath = "model.h5"
ckpt = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

image_size = (28, 28)

datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = False,
                                   validation_split=0.3)

In [3]:
training_set = datagen.flow_from_directory('../[Data] HandWriting-Recognition/MathSymbols',
                                                 target_size = image_size,
                                                 color_mode='grayscale',
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset='training')

Found 139612 images belonging to 17 classes.


In [4]:
validation_set = datagen.flow_from_directory('../[Data] HandWriting-Recognition/MathSymbols',
                                                 target_size = image_size,
                                                 color_mode='grayscale',
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset='validation')

Found 59825 images belonging to 17 classes.


In [5]:
def cnn(image_size):
    classifier = Sequential()
    classifier.add(Conv2D(32, (3, 3), input_shape = (*image_size, 1), activation = 'relu'))
    classifier.add(Dropout(0.2))
    classifier.add(Flatten())
    classifier.add(Dense(units = 128, activation = 'relu'))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(units = 17, activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    classifier.summary()
    return classifier

In [ ]:
neuralnetwork = cnn(image_size)


neuralnetwork.summary()


neuralnetwork.fit_generator(training_set,
                         steps_per_epoch = 139612,
                         epochs = 50,
                         validation_data = validation_set,
                         validation_steps = 59825, 
                         callbacks=[es, ckpt])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
dropout_1 (Dropout)          (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)      

D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.101728). Check your callbacks.
  % delta_t_median)


   488/139612 [..............................] - ETA: 75:21:10 - loss: 1.5364 - acc: 0.5159

D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.107712). Check your callbacks.
  % delta_t_median)


  3105/139612 [..............................] - ETA: 61:08:55 - loss: 0.7374 - acc: 0.7771

D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.101228). Check your callbacks.
  % delta_t_median)


  3107/139612 [..............................] - ETA: 61:10:40 - loss: 0.7371 - acc: 0.7771

In [ ]:
                         
# Predictions

import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.models import load_model

neuralnetwork = load_model('model.h5')

test_image = image.load_img(f'/Test Images/{image_path}', target_size = (90, 60))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
class_index = neuralnetwork.predict(test_image)[0][0]
probability = neuralnetwork.predict_proba(test_image)[0][0]